In [2]:
import numpy as np

In [41]:

# Read binary file
with open('A-EOG_raw_data_example.bin', "rb") as f:
    byte_data = f.read()  # Read the entire file as bytes


# Function to swap adjacent byte pairs
def swap_adjacent_pairs(hex_str):
    return "".join([hex_str[i:i+2] for i in range(0, len(hex_str), 2)][::-1])
    

# Convert bytes to a hexadecimal string
hex_string_ori = byte_data.hex()
print('----raw data')
print(hex_string_ori[6+8*12:28*8])
hex_string = hex_string_ori[6+8*12:]

chunk_size = 8  # 8 hex characters = 32 bits

ori_chunk = [hex_string[i:i+chunk_size] for i in range(0, len(hex_string), chunk_size)]
print('----Original chunk:')
print(ori_chunk[:14])

# Apply the function to each element in the list
chunks = [swap_adjacent_pairs(item) for item in ori_chunk]
print('----Reversed order of each chunk:')
print(chunks[:14])

print(len(chunks))
print(type(chunks))

# synchronization
# 1. detect 8 digits of a
# 2. locate the beginning of the string
# 3. reorder the every two digits in each chunk element to find the timestamp (8 digits before aaaaaaaa)



----raw data
6a02010006c6000105bd0002a1ca0003f1b6000440c40005d90e4a05aaaaaaaa5c1801081abd0009dec3000a6adf000be8cc000c8dbb000d6d02010003
----Original chunk:
['6a020100', '06c60001', '05bd0002', 'a1ca0003', 'f1b60004', '40c40005', 'd90e4a05', 'aaaaaaaa', '5c180108', '1abd0009', 'dec3000a', '6adf000b', 'e8cc000c', '8dbb000d']
----Reversed order of each chunk:
['0001026a', '0100c606', '0200bd05', '0300caa1', '0400b6f1', '0500c440', '054a0ed9', 'aaaaaaaa', '0801185c', '0900bd1a', '0a00c3de', '0b00df6a', '0c00cce8', '0d00bb8d']
815604
<class 'list'>


In [69]:
def hex_to_signed_int(hex_str):
    int_value = int(hex_str, 16)
    if int_value >= 2**23:  # If the value is negative in two’s complement
        int_value -= 2**24
    return int_value
print(chunks[0][2:])


def cal_vol_from_digit(chunks):
    vol_chunks = []
    count=0
    for chunk_element in chunks:
        signed_int = hex_to_signed_int(chunk_element[2:])  # Example: -1
        # print(signed_int)  # Output: -1
        signed_vol = signed_int* 5.00 / (2**(24-1)-1)
        vol_chunks.append(signed_vol)
        # print(signed_vol)
        # print('unit(vol)')
        # * 5.00 / (2^(24-1)-1)
        count = count+1
        if count%100==0:
            print(f'processing {count}/{len(chunks)}')
        
    return vol_chunks

vol_chunks = cal_vol_from_digit(chunks)

01026a
processing 100/815604
processing 200/815604
processing 300/815604
processing 400/815604
processing 500/815604
processing 600/815604
processing 700/815604
processing 800/815604
processing 900/815604
processing 1000/815604
processing 1100/815604
processing 1200/815604
processing 1300/815604
processing 1400/815604
processing 1500/815604
processing 1600/815604
processing 1700/815604
processing 1800/815604
processing 1900/815604
processing 2000/815604
processing 2100/815604
processing 2200/815604
processing 2300/815604
processing 2400/815604
processing 2500/815604
processing 2600/815604
processing 2700/815604
processing 2800/815604
processing 2900/815604
processing 3000/815604
processing 3100/815604
processing 3200/815604
processing 3300/815604
processing 3400/815604
processing 3500/815604
processing 3600/815604
processing 3700/815604
processing 3800/815604
processing 3900/815604
processing 4000/815604
processing 4100/815604
processing 4200/815604
processing 4300/815604
processing 44

ValueError: invalid literal for int() with base 16: ''

input:
.bin data

output:
- vol, rotation (rad/degree),



formula: 
calculation of each angle = linear combination of channels

- aCh1+bCh2+cCh3+dCh4+eCh5+ +Ch8+cCh9 ... +dCh14


synchronization
1. detect 8 digits of a
2. locate the beginning of the string
3. reorder the every two digits in each chunk element to find the timestamp (8 digits before aaaaaaaa)

calibration:
linear scaling.
- manual input of scaling factor
- automatic estimation of the factor
  
considerations:
- how to synchronize between illustration and recording.
- the scaling factor is different across two eyes.
   - scaling factor per angle (?)
   - 4 cali points (ref1, ref2, ref3, ref4), (actual1,.. actual4) -> how to combine the set of angles?
  
- Currently the timestamp starts from the hardware